In [1]:
import cv2
import numpy as np

In [2]:
# def empty(a):
#     pass

# cv2.namedWindow("TrackBars")
# cv2.resizeWindow("TrackBars", 640, 240)
# cv2.createTrackbar("Hue Min", "TrackBars", 127, 179,empty)
# cv2.createTrackbar("Hue Max", "TrackBars", 179, 179,empty)
# cv2.createTrackbar("Sat Min", "TrackBars", 65, 255,empty)
# cv2.createTrackbar("Sat Max", "TrackBars", 255, 255,empty)
# cv2.createTrackbar("Val Min", "TrackBars", 88, 255,empty)
# cv2.createTrackbar("Val Max", "TrackBars", 255, 255,empty)

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(10, 200)

True

In [4]:
#pink, orange, yellow, blue
myColors = [[127, 65, 88, 179, 255, 255],
            [0, 92, 38, 12, 151, 255],
            #[0, 75, 145, 179, 121, 255],
            [80, 36, 0, 117, 255, 255]
           ]
myColorValues = [[153, 51, 255],
                 [0, 51, 255],
                 #[0, 230, 230],
                 [255, 198, 26]]

In [5]:
def findColour(img, myColors, myColorValues):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    count = 0
    newPoints = []
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(img_hsv, lower, upper)
        x, y = getContours(mask)
        cv2.circle(imgResult, (int(x),int(y)), 10, myColorValues[count], cv2.FILLED)
        if x!=0 and y!=0:
            newPoints.append([x, y, count])
        count+=1
#         cv2.imshow(str(color[1]), mask)
    return newPoints

In [6]:
def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
            #cv2.drawContours(imgResult, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            x, y, w, h = cv2.boundingRect(approx)
    return x+w/2, y

In [7]:
def draw(myPoints, myColorValues):
    for point in myPoints:
        cv2.circle(imgResult, (int(point[0]), int(point[1])), 10, myColorValues[point[2]], cv2.FILLED)

In [8]:
myPoints=[]
while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    imgResult = img.copy()
    newPoints = findColour(img, myColors, myColorValues)
    
    if len(newPoints)!=0:
        for newP in newPoints:
            myPoints.append(newP)
    if len(myPoints)!=0:
        draw(myPoints, myColorValues)
#     img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     h_min = cv2.getTrackbarPos("Hue Min", "TrackBars")
#     h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
#     s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
#     s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
#     v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
#     v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
#     lower = np.array([h_min, s_min, v_min])
#     upper = np.array([h_max, s_max, v_max])
#     mask = cv2.inRange(img_hsv, lower, upper)
#     img_results = cv2.bitwise_and(img, img, mask=mask)
    
    cv2.imshow("Image", imgResult)
#     cv2.imshow("Image_hsv", img_hsv)
#     cv2.imshow("Mask", mask)
#     cv2.imshow("img_results", img_results)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()